## SOTTOFASE 1 – Caricamento e Ispezione del Dataset
Obiettivo: caricare e verificare la struttura del file Calendario2324.csv

In [27]:
# 1. Import delle librerie
import pandas as pd

# 2. Caricamento del file CSV
file_path = "Calendario2324.csv"
df = pd.read_csv(file_path)

# 3. Visualizza le prime righe del dataset
print("Prime righe del file:")
display(df.head())

# 4. Verifica delle colonne presenti
print("Colonne presenti nel dataset:")
print(df.columns.tolist())

# 5. Verifica dei tipi di dati
print("\n Tipi di dati per ciascuna colonna:")
print(df.dtypes)


Prime righe del file:


,Giornata,Casa,Ospite,Risultato,Marcatori Casa,Marcatori Trasferta
0,1,Empoli,Hellas Verona,0-1,[],"[{'Giocatore': 'Bonazzoli', 'Minuto': 75, 'Rigore': False, 'Autogol': False}]"
1,1,Frosinone,Napoli,1-3,"[{'Giocatore': 'Harroui', 'Minuto': 7, 'Rigore': True, 'Autogol': False}]","[{'Giocatore': 'Politano', 'Minuto': 24, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Osimhen', 'Minuto': 42, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Osimhen', 'Minuto': 79, 'Rigore': False, 'Autogol': False}]"
2,1,Genoa,Fiorentina,1-4,"[{'Giocatore': 'Biraschi', 'Minuto': 58, 'Rigore': False, 'Autogol': False}]","[{'Giocatore': 'Biraghi', 'Minuto': 5, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Bonaventura', 'Minuto': 11, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Gonzalez', 'Minuto': 40, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Mandragora', 'Minuto': 56, 'Rigore': False, 'Autogol': False}]"
3,1,Inter,Monza,2-0,"[{'Giocatore': 'Lautaro Martinez', 'Minuto': 8, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Lautaro Martinez', 'Minuto': 76, 'Rigore': False, 'Autogol': False}]",[]
4,1,Roma,Salernitana,2-2,"[{'Giocatore': 'Belotti', 'Minuto': 17, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Belotti', 'Minuto': 82, 'Rigore': False, 'Autogol': False}]","[{'Giocatore': 'Candreva', 'Minuto': 36, 'Rigore': False, 'Autogol': False}, {'Giocatore': 'Candreva', 'Minuto': 49, 'Rigore': False, 'Autogol': False}]"


Colonne presenti nel dataset:
['Giornata', 'Casa', 'Ospite', 'Risultato', 'Marcatori Casa', 'Marcatori Trasferta']

 Tipi di dati per ciascuna colonna:
Giornata                int64
Casa                   object
Ospite                 object
Risultato              object
Marcatori Casa         object
Marcatori Trasferta    object
dtype: object


##Sottofase 2 – Parsing dei marcatori.

Obiettivo sottofase 2
Estrarre, per ogni partita:
* la giornata
* il nome di ogni giocatore che ha segnato
* il numero di gol segnati (contando doppiette, triplette, ecc.)

Salvando tutto in una struttura tipo:

* {('lautaro martinez', 1): 2, ...}

In [28]:
import ast
from collections import defaultdict

# 1. Dizionario per contare i gol: chiave = (giocatore, giornata), valore = n gol
gol_per_giocatore = defaultdict(int)

# 2. Funzione di parsing per ciascuna lista di marcatori
def processa_marcatori(lista_marcatori, giornata):
    if pd.isna(lista_marcatori):
        return
    try:
        marcatori = ast.literal_eval(lista_marcatori)
        for marcatore in marcatori:
            if not marcatore.get('Autogol', False):  # Esclude autogol
                nome = marcatore['Giocatore'].strip().lower()
                gol_per_giocatore[(nome, giornata)] += 1
    except Exception as e:
        print(f"Errore parsing giornata {giornata}: {e}")

# 3. Iterazione su ogni riga del dataset
for _, row in df.iterrows():
    giornata = int(row['Giornata'])
    processa_marcatori(row['Marcatori Casa'], giornata)
    processa_marcatori(row['Marcatori Trasferta'], giornata)

# 4. Visualizzazione dei primi risultati
print("Gol per giocatore-giornata (primi 10):")
for k, v in list(gol_per_giocatore.items())[:10]:
    print(f"{k}: {v}")


Gol per giocatore-giornata (primi 10):
('bonazzoli', 1): 1
('harroui', 1): 1
('politano', 1): 1
('osimhen', 1): 2
('biraschi', 1): 1
('biraghi', 1): 1
('bonaventura', 1): 1
('gonzalez', 1): 1
('mandragora', 1): 1
('lautaro martinez', 1): 2


Visualizza i gol delle prime 3 giornate


In [29]:
print("Gol segnati nelle giornate 1, 2 e 3:\n")
for (giocatore, giornata), gol in sorted(gol_per_giocatore.items(), key=lambda x: (x[0][1], x[0][0])):
    if giornata in [1, 2, 3]:
        print(f"Giornata {giornata} - {giocatore.title()}: {gol} gol")


Gol segnati nelle giornate 1, 2 e 3:

Giornata 1 - Almqvist: 1 gol
Giornata 1 - Belotti: 2 gol
Giornata 1 - Biraghi: 1 gol
Giornata 1 - Biraschi: 1 gol
Giornata 1 - Bonaventura: 1 gol
Giornata 1 - Bonazzoli: 1 gol
Giornata 1 - Candreva: 2 gol
Giornata 1 - Chiesa: 1 gol
Giornata 1 - De Ketelaere: 1 gol
Giornata 1 - Di Francesco: 1 gol
Giornata 1 - Giroud: 1 gol
Giornata 1 - Gonzalez: 1 gol
Giornata 1 - Harroui: 1 gol
Giornata 1 - Immobile: 1 gol
Giornata 1 - Lautaro Martinez: 2 gol
Giornata 1 - Mandragora: 1 gol
Giornata 1 - Osimhen: 2 gol
Giornata 1 - Politano: 1 gol
Giornata 1 - Pulisic: 1 gol
Giornata 1 - Rabiot: 1 gol
Giornata 1 - Vlahovic: 1 gol
Giornata 1 - Zortea: 1 gol
Giornata 2 - Aouar: 1 gol
Giornata 2 - Colpani: 2 gol
Giornata 2 - Di Lorenzo: 1 gol
Giornata 2 - Dia: 1 gol
Giornata 2 - Duda: 1 gol
Giornata 2 - Dumfries: 1 gol
Giornata 2 - Duncan: 1 gol
Giornata 2 - Ferguson: 1 gol
Giornata 2 - Giroud: 2 gol
Giornata 2 - Gonzalez: 1 gol
Giornata 2 - Harroui: 1 gol
Giornata 2 -

## Sottofase 3 – Costruzione della matrice Giocatore-Giornata.

Obiettivo:

Creare una tabella dove:
* le righe sono i giocatori
* le colonne sono le giornate: G1, G2, ..., G38
* le celle contengono i gol segnati (0 se nessun gol)

In [30]:
# 1. Estrai tutti i giocatori e giornate dal dizionario
giocatori = sorted(set([nome for (nome, _) in gol_per_giocatore.keys()]))
giornate = sorted(set([g for (_, g) in gol_per_giocatore.keys()]))

# 2. Crea dataframe vuoto con righe = giocatori, colonne = G1 ... G38
colonne = [f"G{g}" for g in range(1, 39)]
df_gol = pd.DataFrame(0, index=giocatori, columns=colonne)

# 3. Riempimento del dataframe con i gol
for (giocatore, giornata), gol in gol_per_giocatore.items():
    colonna = f"G{giornata}"
    df_gol.at[giocatore, colonna] = gol

# 4. Reset index per avere "Giocatore" come colonna
df_gol = df_gol.reset_index().rename(columns={"index": "Giocatore"})

# 5. Visualizza le prime righe
print(" Matrice Giocatore-Giornata (prime righe):")
display(df_gol.head())


 Matrice Giocatore-Giornata (prime righe):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,a. bastoni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,a. carboni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,abraham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,acerbi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,adli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Per vedere il risultato della matrice


In [31]:
# Mostra tutte le righe e tutte le colonne della matrice
pd.set_option('display.max_rows', None)       # Nessun limite di righe
pd.set_option('display.max_columns', None)    # Nessun limite di colonne
pd.set_option('display.width', None)          # Estende la larghezza del display
pd.set_option('display.max_colwidth', None)   # Nessun limite alla larghezza delle colonne

# Visualizza tutta la matrice
display(df_gol)


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,a. bastoni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,a. carboni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,abraham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,acerbi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,adli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,alex sandro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,almqvist,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,aouar,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8,arnautović,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2
9,arthur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


## Sottofase 4 – Gestione dei giocatori multipli / omonimie.

Obiettivo:

Assicurarci che nella matrice:

* Non ci siano duplicati di nomi scritti in modo diverso (es. "Lautaro Martinez" vs "lautaro martinez ").
* Tutti i nomi siano normalizzati, cioè:
* senza spazi iniziali/finali
* tutti in minuscolo o formattati correttamente (es. .title())

In [32]:
# 1. Normalizza tutti i nomi dei giocatori nella colonna "Giocatore"
df_gol['Giocatore'] = df_gol['Giocatore'].apply(lambda x: x.strip().lower())

# 2. Raggruppa per nome normalizzato (se ci fossero duplicati li somma)
df_gol = df_gol.groupby("Giocatore", as_index=False).sum()

# 3. Riporta i nomi in formato "Nome Cognome" capitalizzato
df_gol["Giocatore"] = df_gol["Giocatore"].apply(lambda x: x.title())

# 4. Visualizza il risultato
print(" Matrice con nomi normalizzati e omonimie gestite:")
display(df_gol.head())


 Matrice con nomi normalizzati e omonimie gestite:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,A. Bastoni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. Carboni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Abraham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Acerbi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Adli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
# Impostazioni per vedere tutto
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Visualizza la matrice completa aggiornata
display(df_gol)


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,A. Bastoni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. Carboni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Abraham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Acerbi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Adli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Alex Sandro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,Almqvist,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Aouar,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8,Arnautović,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2
9,Arthur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


Normalizziamo, rimuoviamo accenti e unifichiamo righe duplicate



In [34]:
import unicodedata

# Funzione di normalizzazione completa del nome
def normalizza_nome(nome):
    # Minuscolo e senza spazi
    nome = nome.strip().lower()
    # Rimozione accenti (es. 'è' → 'e')
    nome = unicodedata.normalize('NFD', nome)
    nome = ''.join(c for c in nome if unicodedata.category(c) != 'Mn')
    return nome

# 1. Applica normalizzazione ai nomi dei giocatori
df_gol['Giocatore'] = df_gol['Giocatore'].apply(normalizza_nome)

# 2. Raggruppa di nuovo e somma i gol
df_gol = df_gol.groupby('Giocatore', as_index=False).sum()

# 3. Riformatta i nomi per renderli leggibili (es. lautaro martinez → Lautaro Martinez)
df_gol['Giocatore'] = df_gol['Giocatore'].apply(lambda x: x.title())

# 4. Visualizza la matrice aggiornata
display(df_gol)


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
0,A. Bastoni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. Carboni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Abraham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Acerbi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Adli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Alex Sandro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,Almqvist,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Aouar,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
8,Arnautovic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2
9,Arthur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


Unifica le righe 53 e 54 sotto “Cheddira"


In [35]:
# 1. Visualizza i nomi alla riga 53 e 54 per conferma
print("Righe 53 e 54 (per verifica):")
display(df_gol.iloc[53:55])

# 2. Prendi i due nomi
nome_1 = df_gol.iloc[53]['Giocatore']
nome_2 = df_gol.iloc[54]['Giocatore']

# 3. Somma riga 1 + riga 2 (escludendo la colonna "Giocatore")
riga_sommata = df_gol.iloc[53, 1:] + df_gol.iloc[54, 1:]

# 4. Crea nuova riga unificata
nuova_riga = pd.DataFrame([["Cheddira"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Rimuovi le vecchie due righe
df_gol = df_gol.drop(index=[53, 54]).reset_index(drop=True)

# 6. Aggiungi la nuova riga unificata
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Riordina per nome
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Visualizza per conferma
print("Matrice aggiornata dopo unione manuale:")
display(df_gol[df_gol["Giocatore"] == "Cheddira"])


Righe 53 e 54 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
53,Cheddira,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,0,0,0,1,0
54,Chedira,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Matrice aggiornata dopo unione manuale:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
53,Cheddira,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,2,0,0,0,0,1,0


 Unifica le righe 26 e 62 sotto “Berardi”


In [36]:
# 1. Visualizza le righe 26 e 61 per verifica
print("Righe 26 e 61 (per verifica):")
display(df_gol.iloc[[26, 61]])

# 2. Estrai i nomi dalle righe 26 e 61
nome_1 = df_gol.iloc[26]['Giocatore']
nome_2 = df_gol.iloc[61]['Giocatore']

# 3. Somma i gol (escludendo la colonna 'Giocatore')
riga_sommata = df_gol.iloc[26, 1:] + df_gol.iloc[61, 1:]

# 4. Crea nuova riga con nome corretto
nuova_riga = pd.DataFrame([["Berardi"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Elimina le due righe vecchie
df_gol = df_gol.drop(index=[26, 61]).reset_index(drop=True)

# 6. Aggiungi la nuova riga unificata
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Riordina il dataframe per nome
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Visualizza la riga finale unificata
print("Riga unificata per Berardi:")
display(df_gol[df_gol["Giocatore"] == "Berardi"])


Righe 26 e 61 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
26,Berardi,0,0,2,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61,D. Berardi,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Riga unificata per Berardi:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
26,Berardi,0,0,2,0,1,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Unifica le righe 107 e 109 in “Gudmundsson”



In [37]:
# 1. Visualizza le righe 107 e 109 per conferma
print("Righe 107 e 109 (per verifica):")
display(df_gol.iloc[[107, 109]])

# 2. Estrai i nomi
nome_1 = df_gol.iloc[107]['Giocatore']
nome_2 = df_gol.iloc[109]['Giocatore']

# 3. Somma i valori (escludendo la colonna "Giocatore")
riga_sommata = df_gol.iloc[107, 1:] + df_gol.iloc[109, 1:]

# 4. Crea nuova riga con nome corretto
nuova_riga = pd.DataFrame([["Gudmundsson"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Rimuovi le vecchie righe
df_gol = df_gol.drop(index=[107, 109]).reset_index(drop=True)

# 6. Aggiungi la riga nuova
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Ordina per nome
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Mostra il risultato
print("Riga finale unificata per Gudmundsson:")
display(df_gol[df_gol["Giocatore"] == "Gudmundsson"])


Righe 107 e 109 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
107,Gudmundsson,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109,Guðmundsson,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0


Riga finale unificata per Gudmundsson:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
107,Gudmundsson,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0


Unifica le righe 106, 178, 183 in “Nicolas Gonzalez”


In [38]:
# 1. Visualizza le righe per verifica
print("Righe 106, 178, 183 (per verifica):")
display(df_gol.iloc[[106, 178, 183]])

# 2. Estrai i nomi
nomi = df_gol.iloc[[106, 178, 183]]['Giocatore'].tolist()

# 3. Somma i valori delle tre righe (escludendo la colonna "Giocatore")
riga_sommata = df_gol.iloc[106, 1:] + df_gol.iloc[178, 1:] + df_gol.iloc[183, 1:]

# 4. Crea la nuova riga con nome corretto
nuova_riga = pd.DataFrame([["Nicolas Gonzalez"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Elimina le righe originali
df_gol = df_gol.drop(index=[106, 178, 183]).reset_index(drop=True)

# 6. Aggiungi la nuova riga
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Ordina il dataframe per nome
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Visualizza la riga finale unificata
print("Riga finale unificata per Nicolas Gonzalez:")
display(df_gol[df_gol["Giocatore"] == "Nicolas Gonzalez"])


Righe 106, 178, 183 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
106,Gonzalez,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
178,N. Gonzalez,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1
183,Nico Gonzalez,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Riga finale unificata per Nicolas Gonzalez:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
181,Nicolas Gonzalez,1,1,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1


Unifica le righe 144 e 197 in “Lorenzo Pellegrini”



In [39]:
# 1. Visualizza le righe 144 e 197 per verifica
print("Righe 144 e 197 (per verifica):")
display(df_gol.iloc[[144, 197]])

# 2. Estrai i nomi
nomi = df_gol.iloc[[144, 197]]['Giocatore'].tolist()

# 3. Somma i valori delle due righe (escludendo "Giocatore")
riga_sommata = df_gol.iloc[144, 1:] + df_gol.iloc[197, 1:]

# 4. Crea una nuova riga con il nome corretto
nuova_riga = pd.DataFrame([["Lorenzo Pellegrini"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Rimuovi le due righe originali
df_gol = df_gol.drop(index=[144, 197]).reset_index(drop=True)

# 6. Aggiungi la riga unificata
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Ordina il dataframe alfabeticamente
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Mostra la riga finale unificata
print("Riga finale unificata per Lorenzo Pellegrini:")
display(df_gol[df_gol["Giocatore"] == "Lorenzo Pellegrini"])


Righe 144 e 197 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
144,Lo. Pellegrini,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
197,Pellegrini,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Riga finale unificata per Lorenzo Pellegrini:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
147,Lorenzo Pellegrini,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0


Unifica le righe 141 e 212 in “Rafael Leao”

In [40]:
# 1. Visualizza le righe 141 e 212 per controllo
print("Righe 141 e 212 (per verifica):")
display(df_gol.iloc[[141, 212]])

# 2. Estrai i nomi originali
nomi = df_gol.iloc[[141, 212]]['Giocatore'].tolist()

# 3. Somma i valori delle due righe (escludendo la colonna "Giocatore")
riga_sommata = df_gol.iloc[141, 1:] + df_gol.iloc[212, 1:]

# 4. Crea una nuova riga con il nome corretto
nuova_riga = pd.DataFrame([["Rafael Leao"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Rimuovi le righe originali
df_gol = df_gol.drop(index=[141, 212]).reset_index(drop=True)

# 6. Aggiungi la riga nuova
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Ordina il dataframe per "Giocatore"
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Mostra la riga finale unificata
print("Riga finale unificata per Rafael Leao:")
display(df_gol[df_gol["Giocatore"] == "Rafael Leao"])


Righe 141 e 212 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
141,Leao,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
212,Rafael Leao,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,1


Riga finale unificata per Rafael Leao:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
211,Rafael Leao,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,1,0,1


Unifica le righe 213 e 223 in “Amir Rrahmani”

In [41]:
# 1. Visualizza le righe 213 e 223 per verifica
print(" Righe 213 e 223 (per verifica):")
display(df_gol.iloc[[213, 223]])

# 2. Estrai i nomi originali
nomi = df_gol.iloc[[213, 223]]['Giocatore'].tolist()

# 3. Somma i valori (escludendo "Giocatore")
riga_sommata = df_gol.iloc[213, 1:] + df_gol.iloc[223, 1:]

# 4. Crea nuova riga unificata con il nome corretto
nuova_riga = pd.DataFrame([["Amir Rrahmani"] + riga_sommata.tolist()], columns=df_gol.columns)

# 5. Elimina le due righe originali
df_gol = df_gol.drop(index=[213, 223]).reset_index(drop=True)

# 6. Aggiungi la riga unificata
df_gol = pd.concat([df_gol, nuova_riga], ignore_index=True)

# 7. Ordina il dataframe per nome giocatore
df_gol = df_gol.sort_values("Giocatore").reset_index(drop=True)

# 8. Visualizza la riga risultante
print(" Riga finale unificata per Amir Rrahmani:")
display(df_gol[df_gol["Giocatore"] == "Amir Rrahmani"])


 Righe 213 e 223 (per verifica):


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
213,Rahmani,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
223,Rrahmani,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


 Riga finale unificata per Amir Rrahmani:


,Giocatore,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38
7,Amir Rrahmani,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Esporta e scarica GolGiocatoriTemporale.csv

In [42]:
# 1. Salva il file CSV
df_gol.to_csv("GolGiocatoriTemporale.csv", index=False)
print("File salvato come GolGiocatoriTemporale.csv")

# 2. Codice per scaricarlo in Colab
from google.colab import files
files.download("GolGiocatoriTemporale.csv")


File salvato come GolGiocatoriTemporale.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Dopodichè abbiamo proceduto manualmente alla normalizzazione dei nomi dei giocatori, prendendo come riferimento quelli presenti nel file in cui sono state previamente aggregate e armonizzate tutte le caratteristiche individuali dei calciatori (StatisticheGiocatori.csv). Questo passaggio si è reso necessario per garantire la coerenza e l’allineamento tra i diversi dataset utilizzati nel progetto.

Un ulteriore intervento manuale ha riguardato la gestione degli autogol: il sistema, infatti, non li riconosceva come eventi validi, assegnando erroneamente il valore zero al giocatore autore dell'autorete. Per correggere questa anomalia, abbiamo sostituito manualmente lo 0 con la lettera "A" nelle celle corrispondenti, così da indicare in modo esplicito la presenza di un autogol.